#### Cargas las librerías

In [19]:
# Load Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
from pathlib import Path
import os
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import tensorflow as tf
from keras_tuner import HyperModel
from keras_tuner.tuners import Hyperband
from regex import B
from tensorflow.keras.optimizers import Adam, Adadelta

from sklearn.model_selection import TimeSeriesSplit

from evaluation._mae import MAE
from evaluation._smape import sMAPE
from evaluation._mape import MAPE
from evaluation._rmse import RMSE

#### Cargar dataset

In [20]:
s_path=Path.cwd()
s_prt_path=s_path.parent
file=os.path.join(s_prt_path,'datasets/XM_H_sinOfe.csv')
df_Data= pd.read_csv(file, index_col=0)
# df_Data.reset_index(inplace=True)

In [21]:
df_Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52584 entries, 01/01/2013 0:00 to 31/12/2018 23:00
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   PrecioB    52584 non-null  float64
 1   Demanda    52584 non-null  float64
 2   DispTer    52584 non-null  int64  
 3   DispNoTer  52584 non-null  int64  
 4   VolUtil    52584 non-null  int64  
 5   Aportes    52584 non-null  int64  
 6   day_cls    52584 non-null  int64  
dtypes: float64(2), int64(5)
memory usage: 3.2+ MB


#### Funciones necesarias para las ejecuciones

##### División de datos

In [22]:
# Dividir los datos en train, validación y test
def split_data(data, indexTotal, train_fraq, test_len=8760):

    #slice the last year of data for testing 1 year has 8760 hours
    test_slice = len(data)-test_len

    test_data = data[test_slice:]
    indexTest=indexTotal[test_slice:]

    train_val_data = data[:test_slice]
    index_val_data=indexTotal[:test_slice]

    #make train and validation from the remaining
    train_size = int(len(train_val_data) * train_fraq)
    
    train_data = train_val_data[:train_size]
    indexTrain=index_val_data[:train_size]

    val_data = train_val_data[train_size:]
    indexVal=index_val_data[train_size:]
    
    return train_data, val_data, test_data,train_val_data, indexTrain, indexVal, indexTest, index_val_data



##### Construcción del dataset

In [23]:
# Crear los datos para entrar al modelo, definición de ventanas
def window_dataset(data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=False, expand_dims=False):
    """ Create a windowed tensorflow dataset
    
    """

    #create a window with n steps back plus the size of the prediction length
    window = n_steps + n_horizon
    
    #expand dimensions to 3D to fit with LSTM inputs
    #creat the inital tensor dataset
    if expand_dims:
        ds = tf.expand_dims(data, axis=-1)
        ds = tf.data.Dataset.from_tensor_slices(ds)
    else:
        ds = tf.data.Dataset.from_tensor_slices(data)
    
    #create the window function shifting the data by the prediction length
    ds = ds.window(window, shift=n_horizon, drop_remainder=True)
    
    #flatten the dataset and batch into the window size
    ds = ds.flat_map(lambda x : x.batch(window))
    #ds = ds.shuffle(shuffle_buffer)
    
    #create the supervised learning problem x and y and batch
    if multi_var:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:, :1]))
    else:
        ds = ds.map(lambda x : (x[:-n_horizon], x[-n_horizon:]))
    
    ds = ds.batch(batch_size).prefetch(1)
    
    return ds

# Construcción del dataset usando las ventanas
def build_dataset(traindata,valdata,testdata,train_val_data,train_fraq=0.8, 
                  n_steps=24*30, 
                  n_horizon=24, 
                  batch_size=256, 
                  shuffle_buffer=500, 
                  expand_dims=False, 
                  multi_var=False):
    """If multi variate then first column is always the column from which the target is contstructed.
    """
    
    tf.random.set_seed(23)
    
    
    train_ds = window_dataset(traindata, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    val_ds = window_dataset(valdata, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    test_ds = window_dataset(testdata, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    train_val_ds = window_dataset(train_val_data, n_steps, n_horizon, batch_size, shuffle_buffer, multi_var=multi_var, expand_dims=expand_dims)
    
    print(f"Prediction lookback (n_steps): {n_steps}")
    print(f"Prediction horizon (n_horizon): {n_horizon}")
    print(f"Batch Size: {batch_size}")
    print("Datasets:")
    print(train_ds.element_spec)
    
    return train_ds, val_ds, test_ds, train_val_ds

##### Parámetros y ejecución

In [24]:
# Definición de parámetros para la ejecución del modelo
def get_params(multivar=True):
    lr=3e-4
    n_steps=72
    n_horizon=24
    if multivar:
        n_features=7
    else:
        n_features=1

    return n_steps, n_horizon, n_features, lr

# Almacenamiento de los datos de las ejecuciones
def cfg_model_run(model, history, test_ds):
    return {"model": model, "history" : history, "test_ds": test_ds}

# Ejecuciones de los modelos con función de ventanado
def run_model(traindata,valdata,testdata,train_val_data, model_name, model_func, model_configs, epochs):
    
    n_steps, n_horizon, n_features, lr = get_params(multivar=True)
    
    train_ds, val_ds, test_ds, train_val_ds = build_dataset(traindata,valdata,testdata,train_val_data,n_steps=n_steps, n_horizon=n_horizon, multi_var=True)

    model = model_func(n_steps, n_horizon, n_features, lr=lr)

    model_hist = model.fit(train_ds, validation_data=val_ds, epochs=epochs)

    model_configs[model_name] = cfg_model_run(model, model_hist, test_ds)
    
    return 'Finalizado'

#### Escalamiento de los datos y división del dataset

In [25]:
#scale
scaler = MinMaxScaler(feature_range=(0, 1))
scalerP=MinMaxScaler(feature_range=(0, 1))

vec_precio=scalerP.fit_transform(df_Data.values[:,0].reshape(-1,1))

indexTotal=df_Data.index

multivar_df = scaler.fit_transform(df_Data)
train_multi, val_multi, test_multi,train_val_multi, indexTrain, indexVal, indexTest, index_val_data = split_data(multivar_df,indexTotal, train_fraq=0.8, test_len=8760)
print("Multivarate Datasets")
print(f"Train Data Shape: {train_multi.shape}")
print(f"Val Data Shape: {val_multi.shape}")
print(f"Test Data Shape: {test_multi.shape}")
print(f"Train Val Data Shape: {train_val_data.shape}")
print(f"Index Val Data Shape: {indexTrain.shape}")
print(f"Nulls In Train {np.any(np.isnan(train_multi))}")
print(f"Nulls In Validation {np.any(np.isnan(val_multi))}")
print(f"Nulls In Test {np.any(np.isnan(test_multi))}")
print('Precio',vec_precio.shape)
print('Todo',df_Data.shape)

Multivarate Datasets
Train Data Shape: (35059, 7)
Val Data Shape: (8765, 7)
Test Data Shape: (8760, 7)
Train Val Data Shape: (43824, 7)
Index Val Data Shape: (35059,)
Nulls In Train False
Nulls In Validation False
Nulls In Test False
Precio (52584, 1)
Todo (52584, 7)


##### Obtener dataset para el modelo

In [26]:
n_steps=72
n_horizon=24

train_ds, val_ds, test_ds, train_val_ds = build_dataset(train_multi, val_multi, test_multi,train_val_multi, n_steps=n_steps, n_horizon=n_horizon,batch_size=train_multi.shape[0],multi_var=True)
print('Example sample shapes')
for idx,(x,y) in enumerate(train_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    break

Prediction lookback (n_steps): 72
Prediction horizon (n_horizon): 24
Batch Size: 35059
Datasets:
(TensorSpec(shape=(None, None, 7), dtype=tf.float64, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float64, name=None))
Example sample shapes
x =  (1457, 72, 7)
y =  (1457, 24, 1)


In [29]:
print('Example sample shapes')
for idx,(x,y) in enumerate(train_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    Xtr= x.numpy()
    Ytr=y.numpy()
    break

for idx,(x,y) in enumerate(val_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    Xval= x.numpy()
    Yval=y.numpy()
    break

for idx,(x,y) in enumerate(test_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    Xts= x.numpy()
    Yts=y.numpy()
    break

for idx,(x,y) in enumerate(train_val_ds):
    print("x = ", x.numpy().shape)
    print("y = ", y.numpy().shape)
    Xtr_v= x.numpy()
    Ytr_v=y.numpy()
    break

Example sample shapes
x =  (1457, 72, 7)
y =  (1457, 24, 1)
x =  (362, 72, 7)
y =  (362, 24, 1)
x =  (362, 72, 7)
y =  (362, 24, 1)
x =  (1823, 72, 7)
y =  (1823, 24, 1)


#### Modelos Optimizando Hyperparametros

##### DNN

CARGA DEL MODELO 

In [30]:
class DNNHyperModel(HyperModel):
    def __init__(self, n_steps, n_horizon, n_features,lr):
        self.n_steps = n_steps
        self.n_horizon = n_horizon
        self.n_features = n_features
        self.lr = lr

    def build(self, hp):
        model = tf.keras.models.Sequential()
        model.add(tf.keras.layers.Flatten(input_shape=(self.n_steps, self.n_features)))

        hp_seed =hp.Int('seed', min_value=1, max_value=1000, step=1)
        tf.random.set_seed(hp_seed)
        # Tune the number of units in the first Dense layer
        # Choose an optimal value between 32-512
        hp_units1 = hp.Int('units1', min_value=32, max_value=256, step=32)
                # Choose an optimal value between 32-256
        hp_units2 = hp.Int('units2', min_value=32, max_value=256, step=32)

        hp_dropout1 =hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)

        hp_dropout2 =hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)
        # Tune the activation function to use in the Dense layers
        hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])

        optimizer = hp.Choice('optimizer', values=['adam', 'adadelta'])

        model.add(tf.keras.layers.Dense(units=hp_units1, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp_dropout1))

        model.add(tf.keras.layers.Dense(units=hp_units2, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp_dropout2))

        model.add(tf.keras.layers.Dense(self.n_horizon))


        if optimizer == 'adam':
            optimizer = Adam(learning_rate=self.lr)
        else:
            optimizer = Adadelta(learning_rate=self.lr)


        model.compile(optimizer=optimizer,
                    loss=hp.Choice('loss', values=['huber', 'mae']),
                    metrics=['mae', 'mape'])

        return model

HYPERPARAMETRIZACIÓN DEL MODELO

In [52]:
from sklearn.model_selection import TimeSeriesSplit
from kerastuner.tuners import RandomSearch


hypermodel = DNNHyperModel(*get_params(multivar=True))

s_prt_path=s_path.parent
path=os.path.join(s_prt_path,'experimental_files')

tscv = TimeSeriesSplit(n_splits=5)

tuner = Hyperband(
    hypermodel,
    objective='MAPE',
    max_epochs=40,
    directory=path,
    project_name='dnn'
)

tuner.search_space_summary()

for train_index, val_index in tscv.split(Xtr_v):
    X_train, X_val = Xtr_v[train_index], Xtr_v[val_index]
    y_train, y_val = Ytr_v[train_index], Ytr_v[val_index]
    tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20)

# Get the optimal hyperparameters
best_hps_dnn = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps_dnn.get('units1')}. The optimal number of units in the second densely-connected
layer is {best_hps_dnn.get('units2')}.The optimal activation function for the Dense layers
is {best_hps_dnn.get('activation')}. The optimal optimizer for the model
is {best_hps_dnn.get('optimizer')}. The optimal loss function for the model
is {best_hps_dnn.get('loss')}.the optimal dropout for the model
is {best_hps_dnn.get('dropout1')}.The optimal dropout for the model
is {best_hps_dnn.get('dropout2')}.The optimal seed for the model
is {best_hps_dnn.get('seed')}.
""")

Reloading Tuner from c:\Users\sebas\Documents\ESP_DATOS\SEMESTRE2\MONOGRAFIA 2\MonografiaEAD-main\MonografiaEAD\experimental_files\dnn\tuner0.json
Search space summary
Default search space size: 8
seed (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 1, 'sampling': 'linear'}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'leaky_relu'], 'ordered': False}
optimizer (Choice)
{'default': 'adam', 'conditions': [], 'values': ['adam', 'adadelta'],

##### LSTM

CARGA DEL MODELO

In [53]:
class LSTMHyperModel(HyperModel):
    def __init__(self, n_steps, n_horizon, n_features,lr):
        self.n_steps = n_steps
        self.n_horizon = n_horizon
        self.n_features = n_features
        self.lr = lr

    def build(self, hp):
        model = tf.keras.models.Sequential()

        hp_seed =hp.Int('seed', min_value=1, max_value=1000, step=1)
        tf.random.set_seed(hp_seed)
        # Tune the number of units in the first Dense layer
        # Choose an optimal value between 32-256
        hp_units1 = hp.Int('units1', min_value=32, max_value=256, step=32)
                # Choose an optimal value between 32-256
        hp_units2 = hp.Int('units2', min_value=32, max_value=256, step=32)
                # Choose an optimal value between 32-256
        hp_units3 = hp.Int('units3', min_value=32, max_value=256, step=32)

        hp_dropout1 =hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)

        hp_dropout2 =hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)
        # Tune the activation function to use in the Dense layers
        hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])

        optimizer = hp.Choice('optimizer', values=['adam', 'adadelta'])

        model.add(tf.keras.layers.LSTM(units=hp_units1, activation=hp_activation, input_shape=(self.n_steps, self.n_features), return_sequences=True))
        model.add(tf.keras.layers.LSTM(units=hp_units2, activation=hp_activation, return_sequences=False))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dropout(hp_dropout1))
        model.add(tf.keras.layers.Dense(units=hp_units3, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp_dropout2))
        model.add(tf.keras.layers.Dense(self.n_horizon))

        if optimizer == 'adam':
            optimizer = Adam(learning_rate=self.lr)
        else:
            optimizer = Adadelta(learning_rate=self.lr)

        model.compile(optimizer=optimizer,
                    loss=hp.Choice('loss', values=['huber', 'mae']),
                    metrics=['mae', 'mape'])

        return model

HYPERPARAMETRIZACIÓN

In [57]:
hypermodel = LSTMHyperModel(*get_params(multivar=True))


s_prt_path=s_path.parent
path=os.path.join(s_prt_path,'experimental_files')

tscv = TimeSeriesSplit(n_splits=5)

tuner = Hyperband(
    hypermodel,
    objective='mape',
    max_epochs=40,
    directory=path,
    project_name='LSTM'
)

tuner.search_space_summary()

for train_index, val_index in tscv.split(Xtr_v):
    X_train, X_val = Xtr_v[train_index], Xtr_v[val_index]
    y_train, y_val = Ytr_v[train_index], Ytr_v[val_index]
    tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20)
# Get the optimal hyperparameters
best_hps_lstm =tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM layer
layer is {best_hps_lstm.get('units1')}. The optimal number of units in the second LSTM layer
layer is {best_hps_lstm.get('units2')}. The optimal number of units in the third Dense layer
layer is {best_hps_lstm.get('units3')}.The optimal activation function for layers
is {best_hps_lstm.get('activation')}. The optimal optimizer for the model
is {best_hps_lstm.get('optimizer')}. The optimal loss function for the model
is {best_hps_lstm.get('loss')}.the optimal dropout for the model
is {best_hps_lstm.get('dropout1')}.The optimal dropout for the model
is {best_hps_lstm.get('dropout2')}.The optimal seed for the model
is {best_hps_lstm.get('seed')}.
""")



Trial 90 Complete [00h 00m 42s]
mape: 152.1175537109375

Best mape So Far: 38.29347610473633
Total elapsed time: 00h 36m 16s

The hyperparameter search is complete. The optimal number of units in the first LSTM layer
layer is 64. The optimal number of units in the second LSTM layer
layer is 32. The optimal number of units in the third Dense layer
layer is 160.The optimal activation function for layers
is tanh. The optimal optimizer for the model
is adam. The optimal loss function for the model
is mae.the optimal dropout for the model
is 0.0.The optimal dropout for the model
is 0.0.The optimal seed for the model
is 272.



##### GRU

CARGA DEL MODELO

In [58]:
class GRUHyperModel(HyperModel):
    def __init__(self, n_steps, n_horizon, n_features,lr):
        self.n_steps = n_steps
        self.n_horizon = n_horizon
        self.n_features = n_features
        self.lr = lr

    def build(self, hp):
        model = tf.keras.models.Sequential()

        hp_seed =hp.Int('seed', min_value=1, max_value=1000, step=1)
        tf.random.set_seed(hp_seed)

        # Tune the number of units in the first Dense layer
        # Choose an optimal value between 32-256
        hp_units1 = hp.Int('units1', min_value=32, max_value=256, step=32)
                # Choose an optimal value between 32-256
        hp_units2 = hp.Int('units2', min_value=32, max_value=256, step=32)
                # Choose an optimal value between 32-256
        hp_units3 = hp.Int('units3', min_value=32, max_value=256, step=32)

        hp_dropout1 =hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)

        hp_dropout2 =hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)
        # Tune the activation function to use in the Dense layers
        hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])

        optimizer = hp.Choice('optimizer', values=['adam', 'adadelta'])

        model.add(tf.keras.layers.GRU(units=hp_units1, activation=hp_activation, input_shape=(self.n_steps, self.n_features), return_sequences=True))
        model.add(tf.keras.layers.GRU(units=hp_units2, activation=hp_activation, return_sequences=False))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dropout(hp_dropout1))
        model.add(tf.keras.layers.Dense(units=hp_units3, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp_dropout2))
        model.add(tf.keras.layers.Dense(self.n_horizon))

        if optimizer == 'adam':
            optimizer = Adam(learning_rate=self.lr)
        else:
            optimizer = Adadelta(learning_rate=self.lr)

        model.compile(optimizer=optimizer,
                    loss=hp.Choice('loss', values=['huber', 'mae']),
                    metrics=['mae', 'mape'])

        return model

HYPERPARAMETRIZACIÓN

In [59]:
hypermodel = GRUHyperModel(*get_params(multivar=True))

s_prt_path=s_path.parent
path=os.path.join(s_prt_path,'experimental_files')

tscv = TimeSeriesSplit(n_splits=5)

tuner = Hyperband(
    hypermodel,
    objective='mape',
    max_epochs=40,
    directory=path,
    project_name='GRU'
)

tuner.search_space_summary()

for train_index, val_index in tscv.split(Xtr_v):
    X_train, X_val = Xtr_v[train_index], Xtr_v[val_index]
    y_train, y_val = Ytr_v[train_index], Ytr_v[val_index]
    tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20)

# Get the optimal hyperparameters
best_hps_gru=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first GRU
layer is {best_hps_gru.get('units1')}. The optimal number of units in the second GRU
layer is {best_hps_gru.get('units2')}. The optimal number of units in the third Dense layer
layer is {best_hps_gru.get('units3')}.The optimal activation function for the layers
is {best_hps_gru.get('activation')}. The optimal optimizer for the model
is {best_hps_gru.get('optimizer')}. The optimal loss function for the model
is {best_hps_gru.get('loss')}.the optimal dropout for the model
is {best_hps_gru.get('dropout1')}.The optimal dropout for the model
is {best_hps_gru.get('dropout2')}.The optimal seed for the model
is {best_hps_gru.get('seed')}.
""")


Trial 90 Complete [00h 00m 44s]
mape: 137.51544189453125

Best mape So Far: 38.50728225708008
Total elapsed time: 00h 34m 37s

The hyperparameter search is complete. The optimal number of units in the first GRU
layer is 96. The optimal number of units in the second GRU
layer is 32. The optimal number of units in the third Dense layer
layer is 160.The optimal activation function for the layers
is tanh. The optimal optimizer for the model
is adam. The optimal loss function for the model
is huber.the optimal dropout for the model
is 0.0.The optimal dropout for the model
is 0.0.The optimal seed for the model
is 600.



RESULTADOS DE MEJOR HYPERPARAMETRIZACIÓN

#### CNN_LSTM

In [60]:
import random
from torch import seed


class LSTMCNNHyperModel(HyperModel):
    def __init__(self, n_steps, n_horizon, n_features,lr):
        self.n_steps = n_steps
        self.n_horizon = n_horizon
        self.n_features = n_features
        self.lr = lr

    def build(self, hp):
        model = tf.keras.models.Sequential()

        hp_seed =hp.Int('seed', min_value=1, max_value=1000, step=1)
        tf.random.set_seed(hp_seed)
        # Tune the number of units in the first Dense layer
        # Choose an optimal value between 32-256
        hp_units1 = hp.Int('units1', min_value=32, max_value=256, step=32)

        hp_units2 = hp.Int('units2', min_value=32, max_value=256, step=32)

        hp_units3 = hp.Int('units3', min_value=32, max_value=256, step=32)

        hp_filter1=hp.Int('filters1', min_value=32, max_value=256, step=32)

        hp_filter2=hp.Int('filters2', min_value=32, max_value=256, step=32)

        hp_dropout1 =hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)

        hp_dropout2 =hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)
        # Tune the activation function to use in the Dense layers
        hp_activation = hp.Choice('activation', values=['relu', 'tanh', 'leaky_relu'])

        optimizer = hp.Choice('optimizer', values=['adam', 'adadelta'])

        model.add(tf.keras.layers.Conv1D(filters=hp_filter1, kernel_size=6, activation=hp_activation, input_shape=(self.n_steps,self.n_features)))
        model.add(tf.keras.layers.MaxPooling1D(2))
        model.add(tf.keras.layers.Conv1D(filters=hp_filter2, kernel_size=3, activation=hp_activation))
        model.add(tf.keras.layers.MaxPooling1D(2))
        model.add(tf.keras.layers.LSTM(units=hp_units1, activation=hp_activation, return_sequences=True))
        model.add(tf.keras.layers.LSTM(units=hp_units2, activation=hp_activation, return_sequences=False))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dropout(hp_dropout1))
        model.add(tf.keras.layers.Dense(units=hp_units3, activation=hp_activation))
        model.add(tf.keras.layers.Dropout(hp_dropout2))
        model.add(tf.keras.layers.Dense(self.n_horizon))

        if optimizer == 'adam':
            optimizer = Adam(learning_rate=self.lr)
        else:
            optimizer = Adadelta(learning_rate=self.lr)

        model.compile(optimizer=optimizer,
                    loss=hp.Choice('loss', values=['huber', 'mae']),
                    metrics=['mae', 'mape'])

        return model

In [63]:
hypermodel = LSTMCNNHyperModel(*get_params(multivar=True))

s_prt_path=s_path.parent
path=os.path.join(s_prt_path,'experimental_files')

tscv = TimeSeriesSplit(n_splits=5)

tuner = Hyperband(
    hypermodel,
    objective='mape',
    max_epochs=40,
    directory=path,
    project_name='CNN-LSTM'
)

tuner.search_space_summary()

for train_index, val_index in tscv.split(Xtr_v):
    X_train, X_val = Xtr_v[train_index], Xtr_v[val_index]
    y_train, y_val = Ytr_v[train_index], Ytr_v[val_index]
    tuner.search(X_train, y_train, validation_data=(X_val, y_val), epochs=20)


# Get the optimal hyperparameters
best_hps_lstm_cnn=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM
layer is {best_hps_lstm_cnn.get('units1')}. The optimal number of units in the second LSTM
layer is {best_hps_lstm_cnn.get('units2')}. The optimal number of units in the third Dense layer
layer is {best_hps_lstm_cnn.get('units3')}.The optimal activation function for the layers
is {best_hps_lstm_cnn.get('activation')}. The optimal optimizer for the model
is {best_hps_lstm_cnn.get('optimizer')}. The optimal loss function for the model
is {best_hps_lstm_cnn.get('loss')}.the optimal dropout for the model
is {best_hps_lstm_cnn.get('dropout1')}.The optimal dropout for the model
is {best_hps_lstm_cnn.get('dropout2')}.The optimal seed for the model
is {best_hps_lstm_cnn.get('seed')}.The optimal filter for the model first CONV1D layer
is {best_hps_lstm_cnn.get('filters1')}.The optimal filter for the model second CONV1D layer
is {best_hps_lstm_cnn.get('filters2')}.
""")



Reloading Tuner from c:\Users\sebas\Documents\ESP_DATOS\SEMESTRE2\MONOGRAFIA 2\MonografiaEAD-main\MonografiaEAD\experimental_files\CNN-LSTM\tuner0.json
Search space summary
Default search space size: 11
seed (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 1000, 'step': 1, 'sampling': 'linear'}
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
filters2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 